# Opening and cleaning out ephys data

Note: Use L1imag/formation environment.

## Setup everything

### Import packages

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import numpy as np
import os
import xarray as xr
import matplotlib.pyplot as plt
import scipy
from scipy import signal
%matplotlib widget

from ephyviewer import mkQApp, MainViewer, TraceViewer

import mbTools

### Choose experiment

In [2]:
theExpe = mbTools.experiment()

Local config file loaded from localConfig.ini
current folder \\10.69.168.1\crnldata\waking\audrey_hay\NPX\interimAnalysis\NPX1\PO\Expe_2024-07-23_16-16-23 contains a config file
\\10.69.168.1\crnldata\waking\audrey_hay\NPX\interimAnalysis\NPX1\PO\Expe_2024-07-23_16-16-23\expeConfig1.ini


FileChooser(path='\\10.69.168.1\crnldata\waking\audrey_hay\NPX\interimAnalysis\NPX1\PO\Expe_2024-07-23_16-16-2…

## Load data

In [3]:
theExpe.analyseExpe_findData(fullSampling=True)
#theExpe.setNumLFPchannels(32)

\\10.69.168.1\crnldata\waking\audrey_hay\NPX\interimAnalysis\NPX1\channelMaps.ini
Mapping found and loaded
{'EMG': [{'canal': '6', 'status': 1}], 'PFC': [{'canal': '21', 'status': 2}, {'canal': '20', 'status': 1}], 'OFC': [{'canal': '19', 'status': 2}, {'canal': '18', 'status': 1}], 'M2': [{'canal': '27', 'status': 2}, {'canal': '26', 'status': 1}], 'M1 -> DOWN States': [{'canal': '16', 'status': 2}, {'canal': '17', 'status': 1}], 'S1fl': [{'canal': '29', 'status': 2}, {'canal': '28', 'status': 1}], 'RSP': [{'canal': '30', 'status': 2}, {'canal': '31', 'status': 1}], 'CA1-1': [{'canal': '1', 'status': 2}, {'canal': '0', 'status': 1}], 'V1': [{'canal': '11', 'status': 2}, {'canal': '10', 'status': 1}], 'CA1-2': [{'canal': '2', 'status': 2}, {'canal': '3', 'status': 1}], 'S1bf': [{'canal': '14', 'status': 2}, {'canal': '15', 'status': 1}], 'mEC': [{'canal': '13', 'status': 2}, {'canal': '12', 'status': 1}]}
********found some .bin files********
data recorded with Bonsai
importing \\10.69

In [ ]:
#TODO: make sure this part is still used
if  'OE_LFP' in theExpe.data and False:
    #TODO: make sampling rate round?
    sampling_rate = theExpe.data['OE_LFP'].sampling_rate #20000
    combined = theExpe.data['OE_LFP'].combineStructures("All")#['M1'])

    coords = {
        'brain_areas' : np.array(theExpe.data['OE_LFP'].channelLabels[:]),
        'duration_rec' : np.arange(0, combined.shape[0]/sampling_rate, 1/sampling_rate)
    }

    # Put in xarray
    xrCombined = xr.DataArray(coords=coords, dims=['duration_rec', 'brain_areas'])
    xrCombined.loc[:,:]  = combined

    # Save datas
    combinedFN = os.path.join(theExpe.interimAnalysisPath,'RawDataChannelExtracted.npy')
    np.save(combinedFN, combined)

    print("LFP data saved as a xarray")
else:
    print("no LFP data to save")

#  Downsample all LFP signals to 1 kz.

In [ ]:
#TODO: fails to save with big array because there is no more space on remote location
new_sampling_rate = 1000 # Hz

DSname = 'RawDataChannelExtractedDS.npy'
combinedDSFN = os.path.join(theExpe.interimAnalysisPath,DSname)
print(combinedDSFN)

if os.path.isfile(combinedDSFN):
    print(f"a file {combinedDSFN} already exists so do nothing. Manually delete the old file before proceding again if you want to overwrite")
else:
    All = theExpe.data['OE_LFP'].signal
    old_sampling_rate = np.round(theExpe.data['OE_LFP'].sampling_rate/1000)*1000 # round to the thousand precision
    print(f"downsampling signal from {old_sampling_rate} Hz to {new_sampling_rate} Hz")

    ratio = int(old_sampling_rate/new_sampling_rate)
    newLen = int(All.shape[0] * new_sampling_rate / old_sampling_rate)
    print(f"signal size will be reduced from {All.shape[0]} to {newLen} points")

    #combinedDS = signal.resample(All, newLen, axis = 0)
    combinedDS = signal.decimate(All, ratio, ftype='fir', axis= 0, zero_phase=True)
    print(f"the resampled signal has new dimensions: {combinedDS.shape}")
    
    np.save(combinedDSFN, combinedDS)
    print(f"the downsampled file is saved at : {combinedDSFN}")

RawDataChannelExtractedDS.npy
downsampling signal from 20000.0 Hz to 1000 Hz
signal size will be reduced from 75283456 to 3764172 points
the resampled signal has new dimensions: (3764173, 32)
the downsampled file is saved at : RawDataChannelExtractedDS.npy


## End of notebook. 
Data is cleaned up and saved. Data processing for different cortical areas on specific notebooks. Next step is WakeRemoving notebook.

Below is for quick filtering, plotting and visualisation to assess data quality. Filtering for data processing is done again in specific notebooks.

Data quality is assessed on a Sample, whose value has to be attributed on the initial cell.

## Filtering 

SWR: 120 - 200 Hz

In [ ]:
Sample = combinedDS

f_CA1 = Sample[:, 3].copy()

# Paramètres de notre filtre :
f_lowcut = 120.
f_hicut = 200.
fs = new_sampling_rate
nyq = 0.5 * fs
N = 6                 # Ordre du filtre
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
print(Wn)

# Création du filtre :
b, a = scipy.signal.butter(N, Wn, 'band')
filt_SWR_CA1 = scipy.signal.filtfilt(b, a, f_CA1)

times = np.arange(0, f_CA1.size/new_sampling_rate, 1./new_sampling_rate)



Spindles: 8 - 16 Hz

In [ ]:
f_PFC = Sample[:, 1].copy()
f_S1 = Sample[:, 2].copy()

# Paramètres de notre filtre :
f_lowcut = 10.
f_hicut = 16.
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
N = 4
# Création du filtre :
b, a = scipy.signal.butter(N, Wn, 'band')
filt_Spind_PFC = scipy.signal.filtfilt(b, a, f_PFC)
filt_Spind_S1 = scipy.signal.filtfilt(b, a, f_S1)

# # Calcul de la reponse en fréquence du filtre
# w, h = signal.freqz(b, a)

# # Tracé de la réponse en fréquence du filtre
# fig, ax = plt.subplots(figsize=(8,5)) 

# ax.plot(0.5*fs*w/np.pi, np.abs(h), 'b')

# ax.set_xlabel('frequency [Hz]')
# ax.set_ylabel('Amplitude [dB]')
# ax.grid(which='both', axis='both')

FFT display

In [ ]:
# Calcul du spectre
f, Pxx_den = signal.welch(f_CA1, fs, nperseg=1024)

# Tracé
fig, ax = plt.subplots(figsize=(10,5)) 
ax.semilogy(f, Pxx_den)   #  plot with log scaling on the y axis
ax.set_xlabel('frequency [Hz]')
ax.set_ylabel('PSD [V**2/Hz]')

In [ ]:
plt.close('all')

# Display. 

Can massively be improved: with Matplotlib

In [ ]:
times_sliced = times[000:200000]
filt_to_display = filt_SWR_CA1[000:200000]-1000
f_CA1_sliced = f_CA1[0000:200000]/2
combined = np.stack((f_CA1_sliced, filt_to_display), axis = 1)
# Tracé du signal filtré

fig, ax = plt.subplots(1,1, figsize=(15,5), layout='constrained') 
ax.plot(times_sliced, combined, 'r')
ax.set_xlabel('Temps [sec]')
ax.set_ylabel('Amplitude')


With ephyviewer. 

In [ ]:
# prepare numpy array for ephyviewer

filt_SWR_CA1_sliced= filt_SWR_CA1[000:200000, np.newaxis]
filt_Spind_PFC_sliced= filt_Spind_PFC[000:200000, np.newaxis]
filt_Spind_S1_sliced= filt_Spind_S1[000:200000, np.newaxis]
filt_sliced = Sample[0000:200000,:]
combined2 = filt_sliced[:,0:2].copy()
intf_sliced = filt_sliced[:,2]
intf_sliced = intf_sliced[:, np.newaxis]
combined2 = np.append(combined2, filt_Spind_PFC_sliced, axis=1)
combined2 = np.append(combined2, intf_sliced, axis=1)
combined2 = np.append(combined2, filt_Spind_S1_sliced, axis=1)
intf_sliced = filt_sliced[:,3]
intf_sliced = intf_sliced[:, np.newaxis]
combined2 = np.append(combined2, intf_sliced, axis=1)
combined2 = np.append(combined2, filt_SWR_CA1_sliced, axis = 1)


In [ ]:
%gui qt 
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined2, sample_rate, t_start, 'Signals')
win.add_view(view1)

#Parameters can be set in script
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

#Run
win.show()